# Programming in lower level

* Sometimes a control engineer may want to make computers to give commands to or read measurements from machines.



* Understanding how computers work *under the hood* would be helpful in such situations.



## Working with bits

* To utilize certain features of microcontrollers, often times we need to turn on or off certain bits of certain memory location.

* The pointer of C/C++ is a suitable tool for this type of task.

* However, the smallest unit of memory that we can change is a byte; hence we may need to first read the current status of the particular byte, change that one bit, and then write the *bit pattern* to that byte.

### Revisiting `struct` and `union`

* `struct` is a group of data.  An array is also a group of data.  Difference is, to access data in `struct`, we would use *field name*s; for arrays, indices.

* Also, *field*s of struct may have different types; for arrays, all same types.

* For each field of struct, we may specify its length in **bits**.

* Followinging example shows a 16bit color information.

``` C++
#include <cstdint>

// Josh Kunz, Bit-field Packing in GCC and Clang, https://jkz.wtf/bit-field-packing-in-gcc-and-clang
struct high_color_tag{
    uint16_t red:5;
    uint16_t green:6;
    uint16_t blue:5;
};
```



* We can see that the sum of bits is `5 + 6 + 5 = 16`; two bytes.

* `struct` would allocate `red`, `green`, and `blue` values as follows.

In [ ]:
# https://stackoverflow.com/questions/35160256/how-do-i-output-lists-as-a-table-in-jupyter-notebook
# http://nbviewer.jupyter.org/github/ipython/ipython/blob/4.0.x/examples/IPython%20Kernel/Rich%20Output.ipynb

import IPython.display as disp

# number of bits
n = 16
nr = 5
ng = 6
nb = 5

disp.display(
    disp.Markdown(
        '\n'.join(
            [
                ' | '.join(str(k) for k in range(n-1, 0-1, -1)),
                '|'.join(':---:' for k in range(1, n+1)),
                ' | '.join(['`b`']*nb + ['`g`']*ng + ['`r`']*nr),
            ],
        )
    )
)



* Following C++ code would show an example.

In [ ]:
%%writefile bit_field_struct.cpp

#include <bitset>
#include <cstdint>
#include <cstdlib>
#include <iomanip>
#include <iostream>


// Josh Kunz, Bit-field Packing in GCC and Clang, https://jkz.wtf/bit-field-packing-in-gcc-and-clang
struct high_color_tag{
    uint16_t red:5;
    uint16_t green:6;
    uint16_t blue:5;
};


union high_color_union_tag{
    struct high_color_tag rgb_struct;
    uint16_t hex;
};


int32_t main(const int32_t argn, const char * argv[]){
    union high_color_union_tag rgb_union;
    const int32_t n = 10;

    int32_t r = 0, g = 0, b = 0, i = 0;
    
    std::cout << "| `r` | `g` | `b` | `hex` |" << '\n';
    std::cout << "|:---:|:---:|:---:|:-----:|" << '\n';

    for(i = 0; n > i; ++i){
        r = rand() % (1 << 5);
        g = rand() % (1 << 6);
        b = rand() % (1 << 5);

        rgb_union.rgb_struct.red = r;
        rgb_union.rgb_struct.green = g;
        rgb_union.rgb_struct.blue = b;

        // https://stackoverflow.com/questions/7349689
        std::bitset<5> b_r(r);
        std::bitset<6> b_g(g);
        std::bitset<6> b_b(b);
        std::bitset<16> b_hex(rgb_union.hex);

        std::cout << "| " << std::hex << std::setw(6) << b_r
                    << " | " << std::setw(6) << b_g
                    << " | " << std::setw(6) << b_b
                    << " | " << std::setw(16) << b_hex
                    << " |\n";
    }

    // std::cout << "sizeof(rgb_union.rgb_struct)" << sizeof(rgb_union.rgb_struct) << '\n';

}



In [ ]:
%%bash
# Detect OS type because OSX may need different options
# https://stackoverflow.com/questions/3466166/how-to-check-if-running-in-cygwin-mac-or-linux/18790824
unameOut="$(uname -s)"
case "${unameOut}" in
    Linux*)     machine=Linux;;
    Darwin*)    machine=Mac;;
    CYGWIN*)    machine=Cygwin;;
    MINGW*)     machine=MinGw;;
    *)          machine="UNKNOWN:${unameOut}"
esac


if [ $machine == "Linux" ]; then
    g++ -Wall -g bit_field_struct.cpp -o ./bit_field_struct -Wa,-adhln=bit_field_struct.s
elif [ "Mac" == $machine ]; then
    # https://stackoverflow.com/questions/10990018/
    clang++ -S -mllvm --x86-asm-syntax=intel bit_field_struct.cpp
    clang++ -Wall -g bit_field_struct.cpp -o bit_field_struct
else
    g++ -Wall -g bit_field_struct.cpp -o ./bit_field_struct.s -S
    g++ -Wall -g bit_field_struct.cpp -o ./bit_field_struct
fi



In [ ]:
# https://stackoverflow.com/questions/4760215/running-shell-command-from-python-and-capturing-the-output
# https://stackoverflow.com/questions/35160256/how-do-i-output-lists-as-a-table-in-jupyter-notebook
import subprocess
import IPython.display as disp

result = subprocess.run(['./bit_field_struct'], stdout=subprocess.PIPE)

disp.display(disp.Markdown(result.stdout.decode()))



In [ ]:
%%bash

rm bit_field_struct.cpp bit_field_struct

